# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [45]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cabal\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## 0.1 Load and clean text data

In [46]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values

doj = pd.read_json("C:/Users/cabal/OneDrive/Desktop/QSS20_S24/problemsets/pset3/pset3_inputdata/combined.json", lines = True)



## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()


id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [49]:
## your code to subset to one press release and take the string
pharma = doj[doj["id"] == "17-1204"]["contents"]
pharma = str(pharma)

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [52]:
def cleanString(input_string):
    # Remove numbers and punctuation, retain only letters and spaces
    cleaned_string = re.sub(r'[^a-zA-Z\s]', '', input_string)
    # Split the string into a list of words
    words_list = cleaned_string.split()
    return words_list

pharmaWordList = cleanString(pharma)
pharmaStringClean = " ". join(pharmaWordList)


In [53]:
# Tokenizes all the words in our clean Pharma string
tokens = word_tokenize(pharmaStringClean) # Generate list of tokens, break apart the sentence into a list of tokens

# Assigns all the words in our tokens a part of speech
tokens_pos = pos_tag(tokens) 

# Creates a map where the key is the adjective and the value is the number of time it appears in our text
adjective_map = {}
for x in tokens_pos:
    if x[1] in "JJJJRJjS":
        if x[0] not in adjective_map:
            adjective_map[x[0]] = 1
        else:
            adjective_map[x[0]] = adjective_map[x[0]] + 1

# Converts our map into a dataFrame

df = pd.DataFrame.from_dict(adjective_map, orient='index', columns=['Frequency'])

# Sorts the dataframe into descending order so we get the top 5 adjectives on top
df["Frequency"].sort_values(ascending = False).iloc[0:5]



former        8
opioid        5
nationwide    4
addictive     3
other         3
Name: Frequency, dtype: int64

## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [54]:
# Uses the original pharma and Spacecy to extract all named entities from the press release.
spacy_pharma = nlp(pharma)

In [55]:
## your code here for part B
# Prints out only the Entities with the LAW tag
for one_tok in spacy_pharma.ents:
    if one_tok.label_ == "LAW":
        print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

Entity: RICO; NER tag: LAW
Entity: the Controlled Substances Act; NER tag: LAW
Entity: RICO; NER tag: LAW


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

The RICO Act, which was initially aimed at mafia-style organized crime, also applies to pharmaceutical kickbacks, because it allows for the prosecution of corporate executives (such as those in the pharmaceutical industry) who benefit from illegal activities orchestrated by their organizations, similar to how crime bosses are held accountable for the crimes of their employees

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [56]:
# Extracts all the entities with the label DATE and adds it to a new list
entitiesDate = []
for one_tok in spacy_pharma.ents:
    if one_tok.label_ == "DATE":
        entitiesDate.append(one_tok)

# Those that contain the word "year" or "years" will be printed
yearList = []
for item in entitiesDate:
    if re.search("year", item.text): # Searches our text for years and prints it if it includes it
        yearList.append(item.text)
        print(item)


last year
20 years
three years
five years
three years


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

*The max sentence the CEO may be charged with is 20 years in prison with 3 years of supervised release*

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [57]:
# We loop through the list of words that contain years and then we search where they are in the pharma text
for item in yearList:    
    match = re.search(item, pharma) # returns true
    span = match.span() # gives us the location where the word appears
    print("..." + pharma[span[0] - 200:span[1] + 200] + "...\n") # prints 200 charachters before the words apears and 200 after



...sing bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain. "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Departm...

...eves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.  The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence ...

...in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of 

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [58]:
## your code here for subsetting
doj_subset = doj[(doj["topics_clean"] == "Civil Rights") | (doj["topics_clean"] == "Hate Crimes") | (doj["topics_clean"] == "Project Safe Childhood")] 
doj_subset.shape

(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [59]:
def sentiment (string):
    # Removes all entities from our text
    spacy_temp = nlp(string)
    entities = spacy_temp.ents # tuples of all our entities
    sent_obj = SentimentIntensityAnalyzer()

    try: # will try to replace all the entities and if for whatever reason it cannot will do the sentiment with the text including the entities
        for item in entities:
            string = re.sub(item.text, "", string)
        # Scores the sentiment of the entire press release using SentimentIntensityAnalyzer
        sentiment = sent_obj.polarity_scores(string) # Polarity scale seems to be a scale of negative, neautral and positve words and gets a compound
    except:
        sentiment = sent_obj.polarity_scores(string) # Polarity scale seems to be a scale of negative, neautral and positve words and gets a compound
    finally:
        return sentiment



In [60]:
%%time
## your code here executing the function
sentimentlist = [sentiment(report) for report in doj_subset["contents"]]

CPU times: total: 1min 11s
Wall time: 1min 45s


C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [61]:
# Creates a new dataframe from my list of dictionaries
sentiment_df = pd.DataFrame(sentimentlist)

doj_subset.reset_index(drop=True, inplace=True)
sentiment_df.reset_index(drop=True, inplace=True)

doj_subset_wscore = pd.concat([doj_subset, sentiment_df], axis=1)

# Sort from highest to  lowest neg score and print the top id, contents and neg columns of the two most neg press releases
doj_subset_wscore = doj_subset_wscore.sort_values(by = "neg", ascending = False).reset_index()

doj_subset_wscore.loc[0:1, ['id', 'contents', "neg"]]


,id,contents,neg
0,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.319
1,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.300


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [62]:
## agg and find the mean compound score by topic
doj_subset_wscore.groupby("topics_clean")["compound"].agg(np.mean)

# Add a sentence
print("We might see a variation of scores because some press releases have more negative words or a more negativly charged tone. ")

topics_clean
Civil Rights             -0.079874
Hate Crimes              -0.935596
Project Safe Childhood   -0.636441
Name: compound, dtype: float64

We might see a variation of scores because some press releases have more negative words or a more negativly charged tone. 


# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [64]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

# Adds our custom code words to the stop words already existing 
list_stopwords = stopwords.words("english")

# combines our stop words and the stop words provided 
list_stopwords_new = list_stopwords + custom_doj_stopwords



In [65]:
## your code defining a text processing function
snow_stemmer = SnowballStemmer(language='english')

def processString(contents):
    # Convert the words to lowercase
    loweredString = contents.lower()

    # Removes stopwords
    nostop_contents = [word for word in wordpunct_tokenize(loweredString) 
                           if word not in list_stopwords_new]

    # Only retains alpha words (so removes digits and punctuation) and Only retains words 4 characters or longer
    # Uses the snowball stemmer from nltk to stem

    noStopAllAlphaOverFour = [snow_stemmer.stem(word) for word in nostop_contents
                     if word.isalpha() and len(word) >= 4 ]

    # Returns a joined preprocessed string
    return " ".join(noStopAllAlphaOverFour)


In [66]:
## your code executing the function and adding our new column
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(processString)

In [67]:
## your code showing the examples

doj_subset_wscore[(doj_subset_wscore["id"] == "16-718") | (doj_subset_wscore["id"] == "16-217")][["id", "contents", "processed_text"]]

,id,contents,processed_text
2,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",nine count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict
688,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-invol

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [68]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(),
        columns=vectorizer.get_feature_names_out()) # added the out from google
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [69]:
# Function that given a subsetted Dataframe will print out the top 5 words
def get_topwords(dfnew):
    dfnew = dfnew.drop(["index"], axis = 1)
    top_terms = dfnew[dfnew.columns[4:]].sum(axis = 0)
    print("Top ten words are: ")
    print(top_terms.sort_values(ascending = False)[:10])


# Compound is word that apears in the contents so if we do not rename our compound to compound_score we get to columns with the same name
doj_subset_wscore = doj_subset_wscore.rename(columns = {"compound" : "compound_score"})

# Create a dtm with metadata id, compound and topics clean
dtm_processed_text = create_dtm(list_of_strings= doj_subset_wscore.processed_text,
                      metadata = doj_subset_wscore[['id', 'compound_score', 'topics_clean']])



# Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)
top_5_percent_dtm = dtm_processed_text[dtm_processed_text["compound_score"] >= dtm_processed_text['compound_score'].quantile(0.95)]
get_topwords(top_5_percent_dtm)

# Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)
bottom_5_percent_dtm = dtm_processed_text[dtm_processed_text["compound_score"]<= dtm_processed_text['compound_score'].quantile(0.05)]
get_topwords(bottom_5_percent_dtm)

# Print the top 10 words for press releases in each of the three topics_clean
for item in dtm_processed_text["topics_clean"].unique():
    print( "\n" + "For " + item)
    df = dtm_processed_text[dtm_processed_text["topics_clean"] == item]
    get_topwords(df)




Top ten words are: 
agreement     173
enforc        130
state         118
ensur         110
disabl        107
communiti      99
settlement     87
student        87
general        84
servic         83
dtype: int64
Top ten words are: 
assault     191
crime       171
victim      164
hate        129
defend      126
offic       119
conspir     105
sentenc     104
charg        97
american     95
dtype: int64

For Hate Crimes
Top ten words are: 
victim      591
crime       557
hate        524
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
dtype: int64

For Civil Rights
Top ten words are: 
offic        637
hous         633
discrimin    616
enforc       544
disabl       532
said         497
feder        479
violat       477
state        452
court        414
dtype: int64

For Project Safe Childhood
Top ten words are: 
child          1022
exploit         701
sexual          572
safe            479
childhood       474
project        

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [70]:
## Step 1: re-tokenize and store in list
text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in
                  doj_subset_wscore["processed_text"]]
# ## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below

corpus_fromdict = [text_raw_dict.doc2bow(one_text) for one_text in text_raw_tokens]
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_fromdict, num_topics=3, id2word=text_raw_dict, passes=15, random_state=100)

topics = ldamodel.print_topics(num_words=15)

# Prints out the top 15 words from each category
print("civil rights: " + str(topics[0]) + "\n")

print("project safe childhood: " + str(topics[1]) + "\n")

print("Hate Crimes: " + str(topics[2]))

civil rights: (0, '0.010*"victim" + 0.009*"charg" + 0.009*"offic" + 0.008*"crime" + 0.008*"defend" + 0.008*"feder" + 0.008*"polic" + 0.007*"sentenc" + 0.007*"indict" + 0.007*"said" + 0.007*"violat" + 0.007*"prosecut" + 0.007*"prison" + 0.006*"enforc" + 0.006*"guilti"')

project safe childhood: (1, '0.019*"child" + 0.012*"exploit" + 0.011*"sexual" + 0.009*"safe" + 0.009*"state" + 0.009*"project" + 0.008*"childhood" + 0.008*"children" + 0.008*"pornographi" + 0.008*"crimin" + 0.007*"feder" + 0.007*"prosecut" + 0.006*"offic" + 0.006*"year" + 0.006*"sentenc"')

Hate Crimes: (2, '0.013*"hous" + 0.010*"american" + 0.008*"said" + 0.008*"discrimin" + 0.007*"african" + 0.007*"disabl" + 0.006*"feder" + 0.006*"court" + 0.006*"today" + 0.006*"victim" + 0.006*"defend" + 0.006*"alleg" + 0.006*"fair" + 0.005*"prosecut" + 0.005*"general"')


In [71]:
lda_display = gensimvis.prepare(ldamodel, corpus_fromdict, text_raw_dict)
pyLDAvis.display(lda_display)

## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

*Press releases with specific, consistent terminology align more accurately, while those discussing overlapping themes, like "victim and charge," tend to have less precise mappings since they can be associated with more than one topic*

In [72]:
## your code here to get doc-level topic probabilities 
topic_probs = [ldamodel.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict]
len(topic_probs) == len(doj_subset_wscore)

True

In [73]:
## your code here to add those topic probabilities to the dataframe
topic_df = pd.DataFrame([[prob for prob, prob in doc] for doc in topic_probs])

topic_df = topic_df.rename(columns = {0: "Topic 1", 1: "Topic 2", 2: "Topic 3"})


doj_subset_wscore = pd.concat([doj_subset_wscore.reset_index(drop=True), topic_df.reset_index(drop=True)], axis=1)
doj_subset_wscore["top_topic"] = topic_df.idxmax(axis=1)


In [74]:
## your code here to summarize the topic proportions for each of the topics_clean 

topic_distribution = pd.crosstab(index = doj_subset_wscore["topics_clean"], columns = doj_subset_wscore["top_topic"], normalize = "index")

# multiply everything by 100 to get percentages
topic_distribution = topic_distribution.mul(100)

print(topic_distribution)



top_topic                 Topic 1    Topic 2    Topic 3
topics_clean                                           
Civil Rights            43.606557  19.344262  37.049180
Hate Crimes             39.837398   0.406504  59.756098
Project Safe Childhood   0.602410  99.397590   0.000000


# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [75]:
# Function to create bigrams
def create_bigram_onedoc(text):
    # Turns my string into a list of individual words
    textList = text.split()
    # using list comprehension we loop through each word in our list and combine it with the word that comes after it
    bigramsList = ['_'.join([textList[i], textList[i+1]]) for i in range(len(textList) - 1)]
    return ' '.join(bigramsList)

doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)

# Print the id, processed_text, and processed_text_bigram columns for press release with id = 16-217
doj_subset_wscore[doj_subset_wscore["id"] == "16-217"][["id", "processed_text", "processed_text_bigrams"]]

,id,processed_text,processed_text_bigrams
688,16-217,reach comprehens settlement agreement citi miami miami polic resolv offic involv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc offic involv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quick investig offic involv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss vote settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_offic offic_involv involv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_offic offic_involv involv_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_offic offic_involv involv_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_offic offic_involv involv_shoot shoot_effect effect_quick quick_investig investig_offic offic_involv involv_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_agreement agreement_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa oro

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [76]:
# Creates a dtm with our new processed text bigrams and keep the ID, topicsClean and compound metadata
dtm_bigram = create_dtm(list_of_strings= doj_subset_wscore.processed_text_bigrams,
                      metadata = doj_subset_wscore[['id', 'topics_clean', 'compound_score']])


dtm_processed_text.shape
dtm_bigram.shape

# Comment on why the bigram matrix has more dimensions than the unigram matrix
print("One of the reasons the bigram matrix has more dimensions is that the number of words in the string \nincreases after we have bigrams. Now since we have more words there are more possible combinations in our matrix")


# Find and print the 10 most prevelant bigrams for each of the three topics_clean
for item in dtm_bigram["topics_clean"].unique():
    print( "\n" + "For " + item)
    df = dtm_bigram[dtm_bigram["topics_clean"] == item]
    get_topwords(df)

(717, 6870)

(717, 72721)

One of the reasons the bigram matrix has more dimensions is that the number of words in the string 
increases after we have bigrams. Now since we have more words there are more possible combinations in our matrix

For Hate Crimes
Top ten words are: 
hate_crime          379
african_american    367
plead_guilti        275
year_prison         161
special_agent       124
racial_motiv        114
thoma_perez         111
grand_juri          101
perez_general        95
said_thoma           91
dtype: int64

For Civil Rights
Top ten words are: 
fair_hous         231
deputi_general    221
princip_deputi    221
vanita_gupta      202
gupta_head        200
general_vanita    199
said_princip      186
unit_state        156
nation_origin     143
consent_decre     128
dtype: int64

For Project Safe Childhood
Top ten words are: 
safe_childhood       474
project_safe         472
child_pornographi    450
child_exploit        281
sexual_exploit       223
exploit_children     200
plead_guilti         197
exp

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

*They seem like different stages of the same crime by the language they are using.*

In [78]:
# EXTRA

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Gets all the processed text from our dataframe
texts = doj_subset_wscore['processed_text']

# Creates a matrix to use
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

cosine_similarities = cosine_similarity(tfidf_matrix)

# Make similarity with itself 0
np.fill_diagonal(cosine_similarities, 0)

# Find indices of the maximum values
indices = np.argpartition(cosine_similarities.flatten(), -4)[-4:]

# Convert 1D indices to 2D coordinates
top_pair_indices = np.column_stack(np.unravel_index(indices, cosine_similarities.shape))

# Get the unique documents involved in these top pairs to avoid duplicates if they cross-match
unique_indices = np.unique(top_pair_indices.flatten())
top_documents = doj_subset_wscore.iloc[unique_indices]

for index in unique_indices:
    print(f"Document Index: {index}")
    print(doj_subset_wscore.iloc[index]['processed_text'])




Document Index: 177
solicit sexual explicit photo video minor distribut internet sentenc month prison follow year supervis releas plead guilti coercion entic minor engag sexual activ act general john cronan crimin david weiss delawar made announc sentenc judg leonard stark delawar justin gulisano emma alexand gulisano newark york charg march plead guilti sept accord admiss made connect plea agreement gulisano victim onlin victim year gulisano began request receiv sexual explicit imag video victim gulisano post sexual explicit video victim pornographi websit download post repost viewer addit pornograph websit eventu victim refus make send addit imag video gulisano respond threaten victim occas threaten post victim imag video internet threaten share imag video victim brother threaten victim life immigr custom enforc homeland secur investig delawar child predat task forc investig lauren britsch crimin child exploit obscen section ceo graham robinson delawar prosecut brought part project s